In [1]:
import warnings
  # Settings the warnings to be ignored
warnings.filterwarnings('ignore')

In [2]:
!git clone https://github.com/Project-MONAI/MONAI.git
!cd MONAI/
!pip install -e '.[all]'

Cloning into 'MONAI'...
remote: Enumerating objects: 34495, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 34495 (delta 118), reused 169 (delta 106), pack-reused 34262
Receiving objects: 100% (34495/34495), 64.51 MiB | 39.77 MiB/s, done.
Resolving deltas: 100% (27399/27399), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [3]:
!pip install "monai[einops]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 6.2 MB/s eta 0:00:00


In [4]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

2023-04-13 03:09:23.459767: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-13 03:09:23.532214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 03:09:24.692944: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
import numpy as np
import nibabel as nib
from tqdm import tqdm
import os
from glob import glob
import shutil

In [7]:
from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
    RandFlipd,
    RandRotate90d,
    RandAffined,
    RandGaussianNoised,
    RandShiftIntensityd
)
from monai.handlers.utils import from_engine
from monai.networks.nets import VNet, UNETR, UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric, MeanIoU
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

print_config()

MONAI version: 1.1.0
Numpy version: 1.22.4
Pytorch version: 2.0.0+cu118
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a2ec3752f54bfc3b40e7952234fbeb5452ed63e3
MONAI __file__: /usr/local/lib/python3.9/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 3.0.2
scikit-image version: 0.19.3
Pillow version: 8.4.0
Tensorboard version: 2.12.1
gdown version: 4.6.6
TorchVision version: 0.15.1+cu118
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.4
pandas version: 1.5.3
einops version: 0.6.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



Setup data directory

In [8]:
# directory = os.environ.get("MONAI_DATA_DIRECTORY2")
# root_dir = tempfile.mkdtemp() if directory is None else directory
root_dir ='/content/temp'
print(root_dir)

/content/temp


# Download dataset

In [9]:
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/Task09_Spleen.tar"
md5 = "410d4a301da4e5b2f6f86ec3ddba524e"

compressed_file = os.path.join(root_dir, "Task09_Spleen.tar")
data_dir = os.path.join(root_dir, "Task09_Spleen")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

Task09_Spleen.tar: 1.50GB [01:02, 25.6MB/s]                            

2023-04-13 03:10:38,065 - INFO - Downloaded: /content/temp/Task09_Spleen.tar


2023-04-13 03:10:41,157 - INFO - Verified 'Task09_Spleen.tar', md5: 410d4a301da4e5b2f6f86ec3ddba524e.
2023-04-13 03:10:41,158 - INFO - Writing into directory: /content/temp.


In [10]:
train_images = sorted(glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_labels = sorted(glob.glob(os.path.join(data_dir, "labelsTr", "*.nii.gz")))
data_dicts = [{"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)]
train_files, val_files = data_dicts[:-9], data_dicts[-9:]


# Preprocessing

In [11]:
# Set deterministic training for reproducibility
set_determinism(seed=0)

In [30]:
# transfromation on the original images for evaluation
val_org_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=(1.5, 1.5, 2.0), mode="bilinear"),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image"], source_key="image"),
    ]
)

val_org_ds = Dataset(data=val_files, transform=val_org_transforms)
val_org_loader = DataLoader(val_org_ds, batch_size=1, num_workers=4)

# transfromation for the outputs
post_transforms = Compose(
    [
        Invertd(
            keys=["pred1","pred2","pred3"],
            transform=val_org_transforms,
            orig_keys="image",
            meta_keys=["pred1_meta_dict", "pred2_meta_dict","pred3_meta_dict",],
            orig_meta_keys="image_meta_dict",
            meta_key_postfix="meta_dict",
            nearest_interp=False,
            to_tensor=True,
            device="cpu",
        ),
        AsDiscreted(keys=["pred1","pred2","pred3"], argmax=True, to_onehot=2),
        AsDiscreted(keys="label", to_onehot=2),
    ]
)

In [13]:
# define the device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [31]:
#Create VNet Model
vnet_model = VNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
).to(device)



In [32]:
#Create UNet Model
unet_model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

In [33]:
#Create UNETR Model
unetr_model = UNETR(
    in_channels=1,
    out_channels=2,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

In [34]:
# dictionary for models and files of their best trained models
models_dict ={
    'vnet': "vnet_best_metric_model.pth",
    'unet':"unet_best_metric_model.pth",
    'unetr':"unetr_best_metric_model.pth"
}

In [35]:
path = '/content/drive/MyDrive/Winter23/deep_learning/final_project/best_models'

In [36]:
# list of models
models = []

for k, v in models_dict.items():
  if k=='vnet':
    model =vnet_model
  elif k== 'unet':
    model = unet_model
  elif k=='unetr':
    model = unetr_model

  model.load_state_dict(torch.load(os.path.join(path, v)))
  model.eval()
  models.append(model)

# evaluation

In [37]:
# metrics
dice_metric = DiceMetric(include_background=False, reduction="mean")
iou_metric = MeanIoU(include_background=False, reduction='mean')

## voting ensamble Evaluation on validation original image spacings

In [43]:

with torch.no_grad():
    for val_data in val_org_loader:
        val_inputs = val_data["image"].to(device)
        sw_batch_size = 4
        roi_size = (160, 160, 160)
        val_data["pred1"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[0])
        val_data["pred2"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[1])
        roi_size = (96, 96, 96)
        val_data["pred3"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[2]) 
        val_data = [post_transforms(i) for i in decollate_batch(val_data)]
        
        model1_output,model2_output,model3_output, val_labels = from_engine(["pred1","pred2","pred3","label"])(val_data)
        ensemble_output = [torch.stack((a,b,c), dim=1) for a,b,c in zip(model1_output, model2_output, model3_output)]
        val_outputs =[torch.mode(a, dim=1)[0] for a in ensemble_output]

        # # compute metric for current iteration
        dice_metric(y_pred=val_outputs, y=val_labels)
        iou_metric(y_pred=val_outputs, y=val_labels)

    # aggregate the final mean dice result
    dice_org = dice_metric.aggregate().item()
    iou_org = iou_metric.aggregate().item()
    # reset the status for next validation round
    dice_metric.reset()
    iou_metric.reset()

print(f"Metric on original image spacing: dice metric: {round(dice_org,3)*100}, iou metric: {round(iou_org,3)*100} ")

Metric on original image spacing: dice metri: 95.3, iou metric: 91.10000000000001 


# VNet Evaluation on validation original image spacings

In [44]:
val_truth=[]
predictions = []
with torch.no_grad():
    for val_data in val_org_loader:
        val_inputs = val_data["image"].to(device)
        sw_batch_size = 4
        roi_size = (160, 160, 160)
        val_data["pred1"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[0])
        val_data["pred2"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[1])
        roi_size = (96, 96, 96)
        val_data["pred3"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[2]) 
        val_data = [post_transforms(i) for i in decollate_batch(val_data)]
        
        model1_output,model2_output,model3_output, val_labels = from_engine(["pred1","pred2","pred3","label"])(val_data)

        # # compute metric for current iteration
        dice_metric(y_pred=model1_output, y=val_labels)
        iou_metric(y_pred=model1_output, y=val_labels)

    # aggregate the final mean dice result
    metric_org = dice_metric.aggregate().item()
    iou_org = iou_metric.aggregate().item()
    # reset the status for next validation round
    dice_metric.reset()
    iou_metric.reset()

print(f"VNet: Metric on original image spacing: dice metric: {round(dice_org,3)*100}, iou metric: {round(iou_org,3)*100} ")

VNet: Metric on original image spacing: dice metri: 95.3, iou metric: 69.39999999999999 


## UNet Evaluation on validation original image spacings

In [45]:

with torch.no_grad():
    for val_data in val_org_loader:
        val_inputs = val_data["image"].to(device)
        sw_batch_size = 4
        roi_size = (160, 160, 160)
        val_data["pred1"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[0])
        val_data["pred2"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[1])
        roi_size = (96, 96, 96)
        val_data["pred3"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[2]) 
        val_data = [post_transforms(i) for i in decollate_batch(val_data)]
        
        model1_output,model2_output,model3_output, val_labels = from_engine(["pred1","pred2","pred3","label"])(val_data)

        # # compute metric for current iteration
        dice_metric(y_pred=model2_output, y=val_labels)
        iou_metric(y_pred=model2_output, y=val_labels)

    # aggregate the final mean dice result
    metric_org = dice_metric.aggregate().item()
    iou_org = iou_metric.aggregate().item()
    # reset the status for next validation round
    dice_metric.reset()
    iou_metric.reset()

print(f"UNet: Metric on original image spacing: dice metric: {round(dice_org,3)*100}, iou metric: {round(iou_org,3)*100} ")

UNet: Metric on original image spacing: dice metri: 95.3, iou metric: 88.1 


## UNETR Evaluation on validation original image spacings

In [46]:

with torch.no_grad():
    for val_data in val_org_loader:
        val_inputs = val_data["image"].to(device)
        sw_batch_size = 4
        roi_size = (160, 160, 160)
        val_data["pred1"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[0])
        val_data["pred2"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[1])
        roi_size = (96, 96, 96)
        val_data["pred3"] = sliding_window_inference(val_inputs, roi_size, sw_batch_size, models[2]) 
        val_data = [post_transforms(i) for i in decollate_batch(val_data)]
        
        model1_output,model2_output,model3_output, val_labels = from_engine(["pred1","pred2","pred3","label"])(val_data)

        # # compute metric for current iteration
        dice_metric(y_pred=model3_output, y=val_labels)
        iou_metric(y_pred=model3_output, y=val_labels)

    # aggregate the final mean dice result
    metric_org = dice_metric.aggregate().item()
    iou_org = iou_metric.aggregate().item()
    # reset the status for next validation round
    dice_metric.reset()
    iou_metric.reset()

print(f"UNETR: Metric on original image spacing: dice metric: {round(dice_org,3)*100}, iou metric: {round(iou_org,3)*100} ")

UNETR: Metric on original image spacing: dice metri: 95.3, iou metric: 81.0 
